<center><h1>Hands-On Reinforcement Learning Applied to Trade Execution Algorithms</h1></center>
<center>
Autor: <cite><a href="https://www.linkedin.com/in/aoteog/">Oteo García, Alberto</a></cite>
</center>
<center>
Autor: <cite><a href="https://www.linkedin.com/in/jesus-sanz/">Sanz del Real, Jesús</a></cite>
</center>

----------------


<center><h1>Resumen</h1></center>

<center>

<p style="width:80%">
    En este notebook se presenta el taller de aprendizaje por refuerzo aplicado a un caso real para finanzas. 
    El objetivo es plantear, desde cero, la resolución del problema de <b>algoritmos de ejecución</b> a través del uso
    de técnicas de aprendizaje por refuerzo. Para ello, durante el taller se ha de completar el entorno básico <b>BestExecutionEnv</b> donde los asistentes desarrollarán las partes claves del
    código para obtener un simulador de mercado que, aunque relativamente naive, será funcional. Posteriormente, los participantes tendrán que aplicar algoritmos clásicos y basados en inteligencia artificial (IA) para la
    resolución del problema. 
</p>
<p style="width:80%">
    En términos sencillos, el problema de <b>algoritmos de ejecución</b> trata sobre la ejecución de órdenes con un tamaño lo suficientemente grande como para que su ejecución suponga un <b><i>'problema'</i></b>, como se ha comentado durante la explicación del caso práctico. La resolución de la ejecución de estas órdenes (órdenes CARE) se puede realizar utilizando diferentes aproximaciónes. En el caso de este taller, queremos optimizar el <b><i>timing</i></b> de nuestras órdenes para minimizar el impacto de mercado. Es decir, buscaremos trocear la orden CARE para comprar (vender) a los menores (mayores) precios de la sesión en la que se ejecuta la orden CARE. Como baseline para saber si nuestra ejecución es buena, o mala, utilizaremos el baseline habitual en el sector: el VWAP slippage.
</p> 
<p style="width:80%">
El VWAP slipagge es la desviación, en puntos básicos, del VWAP del mercado en comparación con el VWAP de nuestro algoritmo. Siendo el VWAP el precio medio ponderado por el volumen:
</p>   
    
$$\frac{\sum_{i=1}^T p_i\cdot v_i}{\sum_{i=1}^Tv_i}$$

</center>

---

<center><h1>Datos</h1></center>

<center>
<p style="width:80%">
Los datos para este taller se han extraido de los activos Repsol y Santander (<i>rep_data.pickle</i> y <i>san_data.pickle</i>). Entre la información extraida contamos con la primera posicion del libro de ordenes tanto el bid como del ask, así como el volumen acumulado, todo ello realizando una agrupación por 5 segundos. Las sesiones extraidas del libro de órdenes están comprendidas entre las 13:00h y las 17:25h. El conjunto de datos esta partido en 3 subconjuntos, para los siguientes días bursátiles:
</p>
    
</center> 
<center>
    <p style="width:35%"><b>Train</b>: Primeros 60 días hábiles de 2018</p>
    <p style="width:35%"><b>Validación</b>: Primeros 30 días hábiles de 2019</p>
    <p style="width:35%"><b>Test</b>: Intervalo de los días hábiles del 31 al 60 de 2019 (30 días)</p>
</center>

<center>
<p style="width:80%">
<b>Adicionalmente</b>, para aquellos que quieran aumentar el grado de datalle, se han añadido otros dos ficheros con información más completa del libro de órdenes y de las ejecuciónes de mercado. El fichero <i>orderbook.pkl</i> contiene las posiciones (precios y volúmenes) de hasta la tercera posición del libro de órdenes, agrupados por 5 segundos; el fichero <i>executions.pkl</i> todas las ejecuciones del mercado. En ambos casos, se trata de información para la compañía Repsol, entre las 13:00h y 17:25h de la sesión. El conjunto de datos esta partido en 3 subconjuntos, para los siguientes días bursátiles:
</p>
    
</center> 
<center>
    <p style="width:35%"><b>Train</b>: Primeros 40 días hábiles de 2019</p>
    <p style="width:35%"><b>Validación</b>: Intervalo del 41 al 60 2019 (20 días)</p>
    <p style="width:35%"><b>Test</b>: Intervalo del 61 al 80 2019 (20 días)</p>
</center>

In [1]:
from collections import deque
import pickle
from typing import Any, List, Sequence, Tuple
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sys.path.append("../src/")

from agents.dqn import DDQNAgent
from report.report import plot_results

2023-09-13 16:59:14.525906: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-09-13 16:59:14.525954: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Datos disponibles: san_data.pickle y rep_data.pickle
with open("../data/san_data.pickle", "rb") as f:
        dict_ = pickle.load(f)
dict_.keys()

dict_keys(['train', 'val', 'test'])

In [3]:
data = dict_["train"]
data

{datetime.date(2018, 1, 2):                       ask1   bid1    vask1    vbid1   last      cumvol
 time                                                                  
 2018-01-02 13:00:00  5.455  5.453   1929.0  10859.0  5.454  14168621.0
 2018-01-02 13:00:05  5.455  5.453  11139.0   2285.0  5.453  14168622.0
 2018-01-02 13:00:10  5.453  5.451   2800.0  29956.0  5.453  14170086.0
 2018-01-02 13:00:15  5.453  5.451   2507.0  23017.0  5.454  14175086.0
 2018-01-02 13:00:20  5.453  5.451   2507.0  27692.0  5.454  14175086.0
 ...                    ...    ...      ...      ...    ...         ...
 2018-01-02 17:24:40  5.452  5.449   9869.0  11915.0  5.450  29462062.0
 2018-01-02 17:24:45  5.452  5.450   3609.0  11090.0  5.451  29466188.0
 2018-01-02 17:24:50  5.453  5.451     31.0   3760.0  5.453  29472152.0
 2018-01-02 17:24:55  5.454  5.451   7448.0   8648.0  5.454  29477152.0
 2018-01-02 17:25:00  5.452  5.450   1100.0  11970.0  5.451  29480640.0
 
 [3181 rows x 6 columns],
 datetime

---
# Entorno

In [ ]:
class BestExecutionEnv:

    def __init__(self, data, look_back=60):
        """Inicialización de la clase del entorno que simula
        el libro de ordenes.
        ----------------------------------------------------
        Input:
            - data: 
                Dataframe con los datos previamente
                agrupados del libro de órdenes.
            
            - look_back: 
                Ventana para la generación de features
                roladas en el instante t=0 del episodio.
                Esta ventana representa el rango máximo para
                la construcción de features.
        ----------------------------------------------------
        Variables Internas:
            - episode_bins:
                Número de bines (steps) del episodio.
            - episode_full_len:
                Es igual a look_back + episode_bins.
            - vol_care:
                Volumen total (en títulos) de la orden care.
            - actions_fn:
                Diccionario con las posibles acciones del agente.
                Las claves acceden a la función que evalúa  la acción
                tomada por el agente.
            - n_actions:
                Número de acciones posibles.
            - n_features:
                Número de características de los estados.
            - episode:
                Dataframe que contiene los steps y estados del episodio.
            - episode_full:
                Es el episode añadiendo el look_back antes del comienzo 
                del episodio.
            - episode_vwap:
                VWAP de mercado al final del episodio.
            - market_ep_vol:
                Volumen (títulos) ejecutado por el mercado en cada bin del episodio.
            - state_pos:
                Número de step en el que nos encontramos.
            - exec_vol:
                Acumulado de títulos ejecutados por el algoritmo.
            - action_hist:
                Lista de acciones tomadas por el algoritmo en cada step.
            - market_vwap_hist:
                Lista de VWAP de mercado en cada step.
            - reward_hist:
                Lista de rewards obtenidas en cada step.
            - price_hist:
                Lista de precios ejecutados en cada step.
            - vol_hist:
                Lista de títulos ejecutados en cada step.                
        """
        
        # Fixed params
        self.data = data
        self.look_back = look_back
        self.episode_bins = None
        self.episode_full_len = None
        self.vol_care = None
        
        self.actions_fn = {
            0: self._do_nothing,
            1: self._agg_action,
        }
        
        self.n_actions = len(self.actions_fn)
        self.n_features = self._detect_num_feat()

        # Data variables
        self.episode = None
        self.episode_full = None
        
        # Env variables
        self.episode_vwap = None
        self.market_ep_vol = None
        self.state_pos = 0
        self.exec_vol = 0
        self.actions_hist = []
        self.algo_vwap_hist = []
        self.market_vwap_hist = []
        self.reward_hist = []
        self.price_hist = []
        self.vol_hist = []
        
    def _detect_num_feat(self):
        """Detecta el número de variables del estado.
        Función necesaria para adaptarse automaticamente a los
        cambios en las variables de observation_builder.
        """
        self._reset_env_episode_params()
        self._generate_episode()
        s = self.observation_builder()
        return s.shape[0]
        
    def _reset_env_episode_params(self):
        """
        Reset del episodio e inicialización de los parámetros.
        Las variables internas vuelven a sus valores originales.
        """
        self.episode_full_len = None
        self.episode = None
        self.episode_full = None
        self.episode_vwap = None
        self.market_ep_vol = None
        self.state_pos = 0
        self.algo_vwap = 0
        self.exec_vol = 0
        self.actions_hist = []
        self.algo_vwap_hist = []
        self.market_vwap_hist = []
        self.reward_hist = []
        self.price_hist = []
        self.vol_hist = []
        
    def _generate_episode_params(self):
        """(1) Se determinan las características de la orden a ejecutar.
        La órden queda definida por: 
         - episode_bins:
             Obtención de un número entero aleatorio [400, 600] 
             con una distribución uniforme.
         - vol_care:
             Obtención del porcentaje de steps en el que hay que 
             ejecutar una órden para cubrir la órden care. 
             vol_care responde a un valor uniforme [0.075, 0.125]
             multiplicado por el número self.episode_bins. 
             Lo convertimos a entero.
        """
        # TODO: Int aleatorio entre 400 y 600 como un objeto numpy
        self.episode_bins = "-----"
        # TODO: Float aleatorio entre 0.075 y 0.125
        pct_bins = "-----"
        # TODO: Int multiplicacion pct_bins y episode_bins
        self.vol_care = "-----"
        
        self.episode_full_len = self.episode_bins + self.look_back
        
        assert self.episode_bins <= 600
        assert self.episode_bins >= 400
        assert self.vol_care <= int(self.episode_bins * 0.125)
        assert self.vol_care >= int(self.episode_bins * 0.075)
        assert isinstance(self.vol_care, int)
        
    def _generate_episode(self):
        """(2) Obtenemos el día y hora en el que comienza el episodio.
        """
        self._generate_episode_params()
        
        lenght_episode = 0
        while lenght_episode != self.episode_full_len:
            # TODO: Selección de un dia entre los posibles.
            # Clue: Usa np.random.choice y los dias data.keys
            selected_day = "-----"
            
            # TODO: Extrae selected_day de data
            data_day = "-----"
            
            # TODO: selecciona una hora de inicio aleatoria
            init_time = "-----"
            
            hour_pos = data_day.index.get_loc(init_time)
            initial_position = hour_pos - self.look_back
            final_position = hour_pos + self.episode_bins
            
            if initial_position < 0:
                continue
            else:
                # TODO: Filtra data_day entre por initial_position y final_position
                self.episode_full = "-----"
                
                # TODO: Filtra data_day entre por hour_pos y final_position
                self.episode = "-----"
                
                lenght_episode = self.episode_full.shape[0]
        
            
    def reset(self) -> np.array:
        """Reinicialización del episodio junto con los parámetros.
        Devuelve la primera observación del nuevo episodio.
        """  
        self._reset_env_episode_params()     
        self._generate_episode()
        self._compute_episode_market_feat()
        
        return self.observation_builder()
    
    def observation_builder(self) -> np.array:
        """(3) Función para la construcción de las observaciones del estado.
            ------------------------------------------------------------
            Default:
                - Primera característica es tiempo restante en porcentaje.
                - Seguna característica es el volumen restante en porcentaje.
        """
        # TODO: Construye el vector con las dos características de la descripción
        # Clue: Utiliza episode_bins, state_pos, exec_vol ,vol_care
        time_left = "-----"
        vol_left = "-----"
        obs  = np.array([time_left, vol_left])
        
        return obs
    
    def _compute_episode_market_feat(self) -> Tuple[float, float]:
        """(4) Cálculo de los valores VWAP y Market Vol del episodio.
        Como no tenemos las ejecuciones de mercado, asumimos que el 
        precio es el mid price de cada step.
        """
        # TODO: Calcula el mid price utilizando ask1 y bid1 de episode
        # Opcional: Utiliza un precio más realista para el mkt VWAP
        mid = "-----"
        # TODO: Calcula market_ep_vol
        self.market_ep_vol = "-----"
        self.market_ep_vol[0] = 0
        # TODO: calcula el volumen acumulado del mercado en todo el episodio
        cum_vol = "-----"
        # TODO: calcula el episode_vwap
        self.episode_vwap = "-----"
        
        return self.episode_vwap, self.market_ep_vol
    
    def _compute_algo_vwap(self) -> float:
        """(8) Cálculo del VWAP del algoritmo hasta el step actual.
        """
        # TODO: Calcula el algo_vwap
        # Clue: utiliza price_hist, vol_hist
        p_arr = "-----"
        v_arr = "-----"
        algo_vwap = "-----"
        return algo_vwap
    
    def _compute_reward(self, price: float, vol: float) -> float:

        # TODO: Establece y devuelve un reward cuando vol == 0
        if vol == 0:
            reward = "-----"
            return reward
        # TODO: Calcula y devuelve el reward cuando vol > 0
        # Clue: Utiliza episode_vwap y price para la reward por defecto
        # Opcional: Utiliza el self y elimina los parámetros de la función
        reward = "-----"
        return reward
    
    def _compute_stop_conditions(self) -> Tuple[bool, bool]:
        """(10) Define las condiciones de parada del episodio
        Return:
            Tiempo agotado, orden completada
        """
        # TODO: Calcula las variables de parada y devuélvelas en el orden apropiado
        is_bins_complete = "-----"
        is_ord_complete = "-----"
        return is_bins_complete, is_ord_complete
    
    def _compute_done_reward(self) -> float:
        """(12)
        """
        # TODO: Free style
        done_reward = 0
        return done_reward
    
    def _agg_action(self) -> float:
        """(7) Acción agresiva de compra de un título a precio de episode['ask1'].
        Devolvemos el reward asociado a esa acción.
        """
        # TODO: obtén el precio de la accion agresiva (ask1) en el state_pos
        price = "-----"
        # TODO: guarda price en price_hist, añade 1 a exec_vol y añade 1 a vol_hist
        "-----"
        vol = 1
        self.exec_vol += "-----"
        "-----"
        
        # TODO: utiliza la función apropiada para calcula el algo_vwap
        algo_vwap = "-----"
        # guarda el algo_vwap en algo_vwap_hist
        "-----"
        # TODO: calcula el reward utilizando la función apropiada
        reward = "-----"
        return reward

    def _do_nothing(self) -> float:
        """(9) No hacer nada y devolvemos el reward asociado a la acción
        """
        # TODO: Repite el proceso de _agg_action
        # Clue: Precio y volumen ejecutado = 0
        price = "-----"
        vol = "-----"
        "-----"
        "-----"
        algo_vwap = "-----"
        "-----"
        reward = "-----"
        
        return reward
    
    def _compute_market_vwap(self) -> float:
        """(5) Cálculo del VWAP del mercado hasta el step actual.
        """
        # TODO: Establece un precio para el vol ejecutado por el mkt en cada step
        # Clue: puedes fijarte en _compute_episode_market_feat
        mid_p = "-----"
        mkt_p = "-----"
        # Calcula todos los vwap del mkt hasta el step actual incluido
        v = "-----"
        p_arr = "-----"
        v_arr = "-----"
        sum_vol = "-----"
        # Si el mkt vol hasta el step == 0, devuelve el último precio hasta el step
        if sum_vol == 0:
            return "-----"
        # Calcula y devuelve el vwap acumulado hasta el step
        market_vwap = "-----"
        return market_vwap
    
    def _compute_done(self) -> bool:
        """(11) Reglas de finalización del episodio.
        """
        # TODO: Calcula las condiciones de parada utilizando la función adecuada
        conditions = "-----"
        is_bins_complete = "-----"
        is_ord_complete = "-----"
        done = "-----"
        # TODO: Devuelve done == True si se cumplen cualquiera de las condiciones
        return done

    def step(self, action) -> Tuple[np.array, float, bool, dict]:
        """ Evalua la acción, calcula la recompensa, devuelve el 
        nuevo estado y si el episodio ha terminado.
        """
        market_vwap = self._compute_market_vwap()
        
        act_fn = self.actions_fn.get(action)
        if act_fn is None:
            raise ValueError(
                f"Invalid action {action}. Valid actions {self.actions_fn.keys()}"
            )

        reward = act_fn()

        self.market_vwap_hist.append(market_vwap)
        self.reward_hist.append(reward)
        
        self.state_pos += 1
        
        done = self._compute_done()
        
        if done:
            reward += self._compute_done_reward()
            return None, reward, done, {}
        
        observation = self.observation_builder()
        
        return np.array(observation), reward, done, {}
    
    def action_sample(self) -> int:
        """(13) Devuelve una acción aleatoria. El valor ha de corresponder 
        con las keys de actions_fn.
        """
        # TODO: Toma una acción aleatoria
        # Opcional: ¿Qué distribución de prob es mejor para la exploración?
        p = "-----"
        action = "-----"
        return action

    def stats_df(self):
        """Información para el gráfico de resultados de la ejecución
        """
        
        my_df = pd.DataFrame(
            {"vwap": self.algo_vwap_hist, "vol": self.vol_hist},
            index=list(self.episode.index)[:len(self.algo_vwap_hist)]
        )
        my_df = my_df.reindex(self.episode.index)
        my_df["vol"] = my_df["vol"].fillna(0)
        my_df["vwap"] = my_df["vwap"].ffill()
            
        
        p = self.episode["ask1"]
        v = self.episode["cumvol"].diff().shift(-1)
        last_v = self.episode_full["cumvol"].diff()[-1]
        v.iloc[-1] = last_v
        market_vwap = (p * v).cumsum() / v.cumsum()
        market_df = pd.DataFrame(
            {"vwap": market_vwap, "vol": v},
            index=v.index
        )
        
        mpx = (self.episode["ask1"] + self.episode["bid1"]) / 2
        
        return my_df, market_df, mpx
        
        

---
<center><h2>TWAP</h2></center>

<center>
<p style="width:80%">
El <b><i>time weighted average price</i></b> (TWAP) es un algoritmo tradicional de negociación basado en el precio medio ponderado por el tiempo, utilizado normalmente para la ejecución de órdenes grandes para reducir el impacto de mercado. Básicamente, se trocea la orden de manera que ejecute las órdenes en un tiempo equidistante durante el tiempo de ejecución seleccionado por el cliente. Puede ser fácil adivinar el patrón de negociación de la estrategia en ejecución si sus órdenes no se modifican de una manera especial, por lo que los parámetros se pueden ajustar o alterar para hacer que la estrategia sea más difícil de rastrear. Las soluciones más comunes son la aleatorización del tamaño de las órdenes y/o el tiempo de envio entre cada una de ellas. Es posible limitar la cantidad para que no exceda un porcentaje definido del volumen de participación, o también evitar agresiones de más de una posición del libro. Esto nos ayuda a minimizar el impacto de las estrategias en el mercado.
</p>  
<p style="width:80%">
Para el objetivo del taller, se pide a los participantes implementar un TWAP sin la necesidad de realizar ninguna modificación. El agente TWAP ha de trocear y ejecutar las órdenes hijas de la orden CARE equidistantes en el tiempo, sin necesidad de incorporar mecanimos para la no detección
</p>

In [ ]:
class TWAP(DDQNAgent):
    def act(self, s):
        # TODO: Configura un TWAP determinista utilizando s
        a = "-----"
        return a

------------------
<center><h2>Parametrización del Problema</h2></center>

<center>
<p style="width:80%">
Una vez definido el entorno que vamos a utilizar como simulador de mercado, necesitamos definir el agente que va a encargarse de aprender a ejecutar las órdenes CARE. El agente es el <b><i>cerebro</i></b> del sistema y puede tener, o no, inteligencia artificial implementada. Por ejemplo, la clase TWAP anteriormente implementada depende únicamente de una regla determinista. Por otro lado, los algoritmos vistos durante el bloque de Aprendizaje por Refuerzo dependerán de la construcción de un modelo de aprendizaje y, por tanto, requerirán la parametrización de estos. Los parámetros e hiperparámetros del modelo dependerán de la aproximación y modelo que elijamos.
</p>
</center>

---
<center >
<p style="width:80%">
    <b>Epsilon</b>: Parámetro de exploración para los algoritmos basados en funciones valor (por ejemplo, los deep Q-learning). Puede tomar valores entre [0,1].
</p> 
</center>
<center>
<p style="width:80%">
    <b>Min Epsilon</b>: Valor mínimo del epsilon por si se quier dejar aleatoriedad durante procesos de entrenamiento muy largos en los que el <b>epsilon</b> decaería hasta cero.
</p> 
</center>
<center>
<p style="width:80%">
    <b>Espilon_decay</b>: Valor numérico mediante el cual se reduce el valor de epsilon cada determinado número de episodios.
</p> 
</center>
<center>
<p style="width:80%">
    <b>Gamma</b>: Término de descuento para el calculo de los retornos acumulados con descuento. Dado que nuestro MDP se define con tareas episódicas gamma podrá tomar valores en el intervalo [0,1].
</p> 
</center>
<center>
<p style="width:80%">
    <b>Alpha</b>: Step-size, término indicando el tamaño de las actualizaciones durante el aprendizaje. Dado que nuestro problema es no estacionario (el sistema evoluciona en el tiempo) el alpha será un valor estático o adaptativo pero nunca un valor no descendente a cero en el tiempo.
</p> 
</center>

<center>
<p style="width:80%">
    <b>Buffer Size</b>: Tamaño de la memoria encargada de obtener un conjunto de datos que aproximadamente sea independiente e identicamente distribuido (i.i.d).
</p> 
</center>

---
<center>
<p style="width:80%">
<b>*Nota*</b>: Los parametros que definen la arquitectura de la red dependen de lo flexible y compleja que se contruyan en los agentes. Los agentes ofrecidos para el taller mantienen la filosofia del mismo y se mantendrán sencillos.
</p> 
</center>
<center>
<p style="width:80%">
    <b>Batch size</b>: Tamaño de los paquetes de datos que se envian al modelo durante su aprendizaje.
</p> 
</center>

<center>
<p style="width:80%">
    <b>Hidden_neurons</b>: Número de neuronas en las capatas ocultas de los modelos.
</p> 
</center>

---
<center>
<p style="width:80%">
    <b>Nepisodes</b>: Número de sessiones de trading (episodios) que se vana realizar para el entrenamiento del modelo.
</p> 
</center>
<center>
<p style="width:80%">
    <b>N_log</b>: Valor numérico indicando cada cuantos episodios se realiza un log de la situación actual del aprendizaje.
</p> 
</center>
<center>
<p style="width:80%">
    <b>Learn_after</b>: Valor numérico indicando cada cuantos steps se realiza un aprendizaje por parte de las redes.
</p> 
</center>

In [ ]:
"""
    Agent Params
"""
# TODO: Selecciona un epsilon inicial para el entrenamiento
epsilon = "-----"
# TODO: Selecciona un min_epsilon para el entrenamiento
min_epsilon = "-----"
# TODO: Selecciona un gamma para el aprendizaje
gamma = "-----"
# TODO: Selecciona un alpha para el aprendizaje
alpha = "-----"
# TODO: Selecciona un buffer_size para el aprendizaje
buffer_size = "-----"
# TODO: Selecciona un batch_size para el aprendizaje
batch_size = "-----"
# TODO: Selecciona el número de nueronas para el modelo
hidden_neurons = "-----"

"""
    Training Params
"""
# TODO: Selecciona el número de episodios
nepisodes = "-----"
n_log = 25
#TODO: Determina el epsilon_decay para el proceso de entrenamiento
epsilon_decay = "-----"
learn_after = batch_size

env = BestExecutionEnv(data, 60)

<center><h3>Inicialización del agente</h3></center>

In [ ]:
agent = "-----"

<center><h3>Recolección de los Episodios para el Buffer</h3></center>
<center>
<p style="width:80%">
Antes de empezar el entrenamiento, para que el modelo tenga experiencias de las que aprender y poder tomar acciones greedy con cierto criterio ha de tener muestras para entrenar. Para ello vamos a llenar el buffer con experiencias siguendo una política estocástica con el objetivo de explorar entorno inicialmente.
</p>

In [ ]:
# En este punto eps es 1 -> actuando random
s = env.reset()
for exps in range(buffer_size):  
    a = agent.act(s)
    s1, r, done, _ = env.step(a)
    agent.experience(s, a, r, s1, done)
    s = s1
    if not exps % 10000:
        print(f'buffer exps: {exps}')
    if done:
        s = env.reset()

#### Train Algo

In [ ]:
agent.set_trainable(True)
learn_counter = 0
history_steps = []
history_rewards = []
history_disc_rewards = []
history_losses = []
for episode in range(nepisodes):
    s = env.reset()
    step = 0
    cum_reward = 0
    dis_cum_reward = 0
    episode_losses = []
    while True:
        a = agent.act(s)
        s1, r, done, _ = env.step(a)
        agent.experience(s, a, r, s1, done)
        learn_counter += 1
        cum_reward += r
        dis_cum_reward += agent.gamma ** step * r
        s = s1
        step += 1
        if not learn_counter % learn_after:
            mse = agent.learn()
        if done:
            agent.epsilon = max([agent.epsilon - epsilon_decay, min_epsilon])
            history_rewards.append(cum_reward)
            history_disc_rewards.append(dis_cum_reward)
            history_losses.append(mse)
            history_steps.append(step)
            if not episode % n_log:
                mse = agent.learn()
                print(
                    f'Episode: {episode}, '
                    f'steps: {np.round(np.mean(history_steps[-n_log:]), 2)}, '
                    f'rew: {np.round(np.mean(history_rewards[-n_log:]), 2)}, '
                    f'mse: {np.round(mse)}, '
                    f'eps: {np.round(agent.epsilon, 2)}'
                )
            break

#### Plot Results

In [ ]:
agent.set_trainable(False)
cum_reward = 0
step = 0
s = env.reset()
while True:
    a = agent.act(s)
    s, r, done, _ = env.step(a)
    step += 1
    cum_reward += agent.gamma ** step * r
    if done:
        break
plot_results(env)

In [ ]:
pd.DataFrame(history_rewards).rolling(20).mean().plot()